In [1]:
import pandas as pd
import matplotlib.pylab as plt
import statsmodels.api as sm
import numpy as np
import seaborn as sns
from scipy import stats

### Raw 데이터 읽기

In [2]:
# 카테고리
read_path = '../csv/categories.csv'
df_category = pd.read_csv(read_path)

df_category = df_category[['category', 'name']]
df_category['category'] = df_category['category'].astype(str)

df_category.tail()

,category,name
1219,240060020,프로그래머
1220,240070040,설문조사
1221,240080060,유아유치원
1222,240100010,매장관리
1223,240100090,기타 업종


In [3]:
# 번프 전체 데이터
read_path = '../csv/bunp_history_all.csv'
bunp_raw = pd.read_csv(read_path)

bunp_raw = bunp_raw.rename(columns={'seller_pid_price': 'total_price'})

# change type to str
for col in ['buyer_uid', 'seller_uid']:
    
    df = bunp_raw
    
    df[col] = df[col].astype('str')

bunp_raw.tail()

,created_at,category,status,buyer_uid,seller_uid,total_price
14334118,2019-06-30 08:44:07,600400003.0,1,3164423,2941858,29900
14334119,2019-06-30 13:13:12,600700003.0,1,4658579,9264834,6000
14334120,2019-06-30 16:34:13,500113011.0,4,1296566,2775958,45000
14334121,2019-06-30 13:44:19,500119001.0,1,558255,6077583,20000
14334122,2019-06-30 12:58:22,800300001.0,1,9177490,4019525,64500


In [4]:
# 2017년 이후 번프 내역
bunp_raw_17 = bunp_raw

# make 'year' column
bunp_raw_17['year'] = bunp_raw_17['created_at'].astype(str).str[:4]
bunp_raw_17['year'] = bunp_raw_17['year'].astype(int)

bunp_raw_17 = bunp_raw_17[bunp_raw_17['year'] >= 2017]

# drop 'year' column
bunp_raw_17 = bunp_raw_17.drop(['year'], axis=1)

bunp_raw_17.tail()

,created_at,category,status,buyer_uid,seller_uid,total_price
14334118,2019-06-30 08:44:07,600400003.0,1,3164423,2941858,29900
14334119,2019-06-30 13:13:12,600700003.0,1,4658579,9264834,6000
14334120,2019-06-30 16:34:13,500113011.0,4,1296566,2775958,45000
14334121,2019-06-30 13:44:19,500119001.0,1,558255,6077583,20000
14334122,2019-06-30 12:58:22,800300001.0,1,9177490,4019525,64500


In [5]:
# 거래관련 상담센터 신고 list
read_path = '../csv/help_reporter_frauder.csv'
help_raw = pd.read_csv(read_path)

help_raw['fraud_help'] = 1

help_raw = help_raw.sort_values(['reporter_uid', 'fraud_uid', 'created_at'])  # sort again, ascending

help_raw = help_raw.drop_duplicates(['reporter_uid', 'fraud_uid'])  # drop duplicated row, keep 1st row

help_raw = help_raw.rename(columns={'created_at': 'created_at_help'})

help_raw['month_at'] = help_raw['created_at_help'].astype(str).str[:7]

# change type to str
for col in ['reporter_uid', 'fraud_uid']:
    
    df = help_raw
    
    df[col] = df[col].astype('str')
    

help_raw.tail()

,discussion_id,created_at_help,reporter_uid,fraud_uid,fraud_help,month_at
197015,838830,2019-07-05 07:56:30,9317104,0,1,2019-07
196983,838715,2019-07-04 23:36:37,9317700,8767099,1,2019-07
196782,837888,2019-07-03 21:12:13,9322294,0,1,2019-07
196784,837893,2019-07-03 21:17:22,9322294,9320707,1,2019-07
196876,838255,2019-07-04 13:55:10,9323927,4623330,1,2019-07


### Merge

In [5]:
# merge
bunp_help = pd.merge(bunp_raw_17, help_raw, 
                            left_on=['buyer_uid', 'seller_uid'],
                            right_on=['reporter_uid', 'fraud_uid'],
                            how='left')


bunp_help['fraud_help'] = bunp_help['fraud_help'].fillna(0)

bunp_help['month_at'] = bunp_help['created_at'].astype(str).str[:7]
bunp_help = bunp_help.drop(['created_at', 'created_at_help', 'discussion_id', 'reporter_uid', 'fraud_uid'], axis=1)  # drop columns

# 1-depth 카테고리 붙이기
bunp_help['category_3'] = bunp_help['category'].astype(str).str[:3]

bunp_help = pd.merge(bunp_help, df_category, left_on='category_3', right_on='category', how='left')

bunp_help = bunp_help.drop(['category_x', 'category_y'], axis=1)  # drop columns

# 10만원 단위 구간
bunp_help['price_range'] = (bunp_help['total_price'] / 100000).apply(np.floor) * 100000

# 5만원 이하
bunp_help['less_50,000'] = bunp_help['total_price'].apply(lambda x: 1 if x <= 50000 else 0)

# 3만원 이하
bunp_help['less_30,000'] = bunp_help['total_price'].apply(lambda x: 1 if x <= 30000 else 0)

bunp_help.tail()

,status,buyer_uid,seller_uid,total_price,fraud_help,month_at,category_3,name,price_range,"less_50,000","less_30,000"
14121676,1,3164423,2941858,29900,0.0,2019-06,600,디지털/가전,0.0,1,1
14121677,1,4658579,9264834,6000,0.0,2019-06,600,디지털/가전,0.0,1,1
14121678,4,1296566,2775958,45000,0.0,2019-06,500,유아동/출산,0.0,1,0
14121679,1,558255,6077583,20000,0.0,2019-06,500,유아동/출산,0.0,1,1
14121680,1,9177490,4019525,64500,0.0,2019-06,800,생활/문구/가구/식품,0.0,0,0


### Aggregate

In [6]:
bunp_help_result = bunp_help.groupby(['month_at', 'category_3', 'name', 'fraud_help', 'price_range', 
                                      'less_50,000', 'less_30,000'
                            ], as_index=False).agg({'total_price': 'sum', 'buyer_uid': 'count'})

bunp_help_result = bunp_help_result.rename(columns={'total_price': 'price_sum', 'buyer_uid': 'trx_count'})

# save
save_path = '../csv/result/bunp_help_result.csv'
bunp_help_result.to_csv(save_path, index=False, mode='w', header=True)

bunp_help_result.tail()

,month_at,category_3,name,fraud_help,price_range,"less_50,000","less_30,000",price_sum,trx_count
16968,2019-06,999,기타,1.0,100000.0,0,0,765000,6
16969,2019-06,999,기타,1.0,200000.0,0,0,600000,3
16970,2019-06,999,기타,1.0,400000.0,0,0,490000,1
16971,2019-06,999,기타,1.0,500000.0,0,0,1550000,3
16972,2019-06,999,기타,1.0,800000.0,0,0,850000,1


### 상담센터에 신고된 건 중 번프에 매칭되는 비율

In [10]:
# 차집합 구하기
help_minus_bunp = pd.merge(bunp_raw_17, help_raw, 
                            left_on=['buyer_uid', 'seller_uid'],
                            right_on=['reporter_uid', 'fraud_uid'],
                            how='outer', indicator=True)

help_minus_bunp = help_minus_bunp[(help_minus_bunp['_merge']=='both') |
                                  (help_minus_bunp['_merge']=='right_only')]

help_minus_bunp = help_minus_bunp.reset_index()[['discussion_id', 
                                                 'created_at', 
                                                 'reporter_uid', 
                                                 'fraud_uid', 
                                                 'fraud_help', 
                                                 'month_at', 
                                                 'total_price',
                                                 '_merge']] 

# replace_values
replace_values = {'botn': 'merged', 'right_only': 'missed'}
help_minus_bunp = help_minus_bunp.replace({'_merge' : replace_values})

help_minus_bunp.tail()

,discussion_id,created_at,reporter_uid,fraud_uid,fraud_help,month_at,total_price,_merge
172318,838830.0,NaN,9317104,0,1.0,2019-07,NaN,missed
172319,838715.0,NaN,9317700,8767099,1.0,2019-07,NaN,missed
172320,837888.0,NaN,9322294,0,1.0,2019-07,NaN,missed
172321,837893.0,NaN,9322294,9320707,1.0,2019-07,NaN,missed
172322,838255.0,NaN,9323927,4623330,1.0,2019-07,NaN,missed


In [12]:
help_minus_bunp_result = help_minus_bunp.groupby(['month_at', '_merge'
                            ], as_index=False).agg({'discussion_id': 'count'})

help_minus_bunp_result = help_minus_bunp_result.rename(columns={'discussion_id': 'help_count'})

# save
save_path = '../csv/result/help_minus_bunp_result.csv'
help_minus_bunp_result.to_csv(save_path, index=False, mode='w', header=True)


help_minus_bunp_result.tail()

,month_at,_merge,help_count
103,2019-05,missed,2505
104,2019-06,both,2048
105,2019-06,missed,2556
106,2019-07,both,195
107,2019-07,missed,513


### 상담센터에 신고된 유저 중 영구 제재된 유저 비율

In [17]:
# 영구 제재 받은 uid list
read_path = '../csv/block_uid.csv'
fraud_uid_block = pd.read_csv(read_path)

fraud_uid_block['block_infinite'] = 1

# change type to str
fraud_uid_block['uid'] = fraud_uid_block['uid'].astype(str)

fraud_uid_block.tail()

,uid,block_infinite
143847,5834292,1
143848,5067006,1
143849,8755369,1
143850,5083812,1
143851,7961342,1


In [5]:
# 2017년 1월 ~ 2019년 6월 상담센터 신고 내역
help_raw_17 = help_raw


help_raw_17['created_at_help'] = pd.to_datetime(help_raw_17['created_at_help'])

help_raw_17['year'] = help_raw_17['created_at_help'].dt.year
help_raw_17['month'] = help_raw_17['created_at_help'].dt.month

help_raw_17 = help_raw_17[help_raw_17['year'] >= 2017]

help_raw_17 = help_raw_17[(help_raw_17['year'] != 2019) | (help_raw_17['month'] <= 6)]

help_raw_17 = help_raw_17.drop(['year', 'month'], axis=1)

help_raw_17.tail()

,discussion_id,created_at_help,reporter_uid,fraud_uid,fraud_help,month_at
195988,834704,2019-06-28 23:35:47,9281092,7344857,1,2019-06
195764,833830,2019-06-27 18:26:33,9282335,3893252,1,2019-06
196064,834966,2019-06-29 14:45:24,9283571,7165014,1,2019-06
196052,834921,2019-06-29 13:32:34,9290388,undefined,1,2019-06
196214,835513,2019-06-30 17:59:49,9300550,8733407,1,2019-06


In [20]:
help_block = pd.merge(help_raw_17, fraud_uid_block, 
                     left_on='fraud_uid', right_on='uid',
                     how='left')

help_block['block_infinite'] = help_block['block_infinite'].fillna(0)

help_block = help_block[['fraud_uid', 'block_infinite']]

help_block = help_block.sort_values(['fraud_uid', 'block_infinite'])  # sort again, ascending

help_block = help_block.drop_duplicates(['fraud_uid', 'block_infinite'])  # drop duplicated row, keep 1st row

help_block.tail()

,fraud_uid,block_infinite
34676,999450,1.0
17237,999606,1.0
48732,999920,0.0
93495,999935,0.0
600,undefined,0.0


In [22]:
help_block_result = help_block.groupby(['block_infinite'], as_index=False).agg({'fraud_uid': 'count'})

help_block_result = help_block_result.rename(columns={'fraud_uid': 'fraud_count'})

# save
save_path = '../csv/result/help_block_result.csv'
help_block_result.to_csv(save_path, index=False, mode='w', header=True)


help_block_result.tail()

,block_infinite,fraud_count
0,0.0,34606
1,1.0,22456


### 판매자의 번프 순서별 신고율

In [6]:
help_raw_order = help_raw

# 테스트 계정 제외
test_account = ['880623', '2080436', '0', 'undefined']
help_raw_order = help_raw_order[~help_raw_order.fraud_uid.isin(test_account)]

help_raw_order['created_at_help'] = pd.to_datetime(help_raw_order['created_at_help'])

help_raw_order['fraud_order'] = help_raw_order.groupby('fraud_uid')['created_at_help'].rank(ascending=True, method='dense')

help_raw_order.tail()


/Users/quicket/python-bunjang/venv/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/quicket/python-bunjang/venv/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,discussion_id,created_at_help,reporter_uid,fraud_uid,fraud_help,month_at,fraud_order
196779,837879,2019-07-03 20:57:43,9313906,7604908,1,2019-07,3.0
196879,838263,2019-07-04 14:01:54,9313910,4660646,1,2019-07,1.0
196983,838715,2019-07-04 23:36:37,9317700,8767099,1,2019-07,1.0
196784,837893,2019-07-03 21:17:22,9322294,9320707,1,2019-07,2.0
196876,838255,2019-07-04 13:55:10,9323927,4623330,1,2019-07,1.0


In [7]:
bunp_raw_order = bunp_raw

# 테스트 계정 제외
test_account = ['880623', '2080436', '0', 'undefined']
bunp_raw_order = bunp_raw_order[~bunp_raw_order.seller_uid.isin(test_account)]

bunp_raw_order['created_at'] = pd.to_datetime(bunp_raw_order['created_at'])

bunp_raw_order['sell_order'] = bunp_raw_order.groupby('seller_uid')['created_at'].rank(ascending=True, method='dense')

bunp_raw_order.head()

/Users/quicket/python-bunjang/venv/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/quicket/python-bunjang/venv/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,created_at,category,status,buyer_uid,seller_uid,total_price,year,sell_order
3,2016-10-10 19:07:28,700200004.0,4,4464665,4008188,1005,2016,3.0
4,2016-10-11 16:04:40,310010020.0,2,1574725,2737871,10000,2016,1.0
5,2016-10-11 16:20:36,310040010.0,3,1574725,2737871,10000,2016,2.0
6,2016-10-12 12:55:22,310030030.0,1,3798733,23159,122121212,2016,8.0
7,2016-10-12 17:30:59,600200001.0,1,1574725,4747757,35550,2016,6.0


In [8]:
# merge
bunp_help_order = pd.merge(bunp_raw_order, help_raw_order, 
                            left_on=['buyer_uid', 'seller_uid'],
                            right_on=['reporter_uid', 'fraud_uid'],
                            how='left')


bunp_help_order['fraud_help'] = bunp_help_order['fraud_help'].fillna(0)
bunp_help_order['fraud_order'] = bunp_help_order['fraud_order'].fillna(0)

bunp_help_order['month_at'] = bunp_help_order['created_at'].astype(str).str[:7]
bunp_help_order = bunp_help_order.drop(['created_at', 'created_at_help', 'discussion_id', 'reporter_uid', 'fraud_uid'], axis=1)  # drop columns

# 1-depth 카테고리 붙이기
bunp_help_order['category_3'] = bunp_help_order['category'].astype(str).str[:3]

bunp_help_order = pd.merge(bunp_help_order, df_category, left_on='category_3', right_on='category', how='left')

bunp_help_order = bunp_help_order.drop(['category_x', 'category_y'], axis=1)  # drop columns

# 10만원 단위 구간
bunp_help_order['price_range'] = (bunp_help_order['total_price'] / 100000).apply(np.floor) * 100000

# 5만원 이하
bunp_help_order['less_50,000'] = bunp_help_order['total_price'].apply(lambda x: 1 if x <= 50000 else 0)

# 3만원 이하
bunp_help_order['less_30,000'] = bunp_help_order['total_price'].apply(lambda x: 1 if x <= 30000 else 0)

# 첫 판매 및 2번째 이상 판매 구분
bunp_help_order['1st_sell'] = bunp_help_order['sell_order'].apply(lambda x: 1 if x == 1 else 0)

# 20번 초과 거래는 같은 그룹으로 매핑
bunp_help_order['sell_order_group'] = bunp_help_order['sell_order'].apply(lambda x: x if x <= 20 else 'more_20th_sell')


bunp_help_order.tail()

,status,buyer_uid,seller_uid,total_price,year,sell_order,fraud_help,month_at,fraud_order,category_3,name,price_range,"less_50,000","less_30,000",1st_sell,sell_order_group
14334019,1,3164423,2941858,29900,2019,30.0,0.0,2019-06,0.0,600,디지털/가전,0.0,1,1,0,more_20th_sell
14334020,1,4658579,9264834,6000,2019,15.0,0.0,2019-06,0.0,600,디지털/가전,0.0,1,1,0,15
14334021,4,1296566,2775958,45000,2019,39.0,0.0,2019-06,0.0,500,유아동/출산,0.0,1,0,0,more_20th_sell
14334022,1,558255,6077583,20000,2019,10.0,0.0,2019-06,0.0,500,유아동/출산,0.0,1,1,0,10
14334023,1,9177490,4019525,64500,2019,162.0,0.0,2019-06,0.0,800,생활/문구/가구/식품,0.0,0,0,0,more_20th_sell


In [9]:
# Aggregate
bunp_help_order_result = bunp_help_order.groupby(['month_at', 'category_3', 'name', 
                                                   'fraud_help', 'price_range',
                                                   'sell_order_group',
                                      'less_50,000', 'less_30,000'
                            ], as_index=False).agg({'total_price': 'sum', 'buyer_uid': 'count'})

bunp_help_order_result = bunp_help_order_result.rename(columns={'total_price': 'price_sum', 
                                                                'buyer_uid': 'trx_count'})
# 300만원 이상 거래 제외
bunp_help_order_result = bunp_help_order_result[bunp_help_order_result['price_range'] < 3000000]

# save
save_path = '../csv/result/bunp_help_result.csv'
bunp_help_order_result.to_csv(save_path, index=False, mode='w', header=True)

bunp_help_order_result.tail()

,month_at,category_3,name,fraud_help,price_range,sell_order_group,"less_50,000","less_30,000",price_sum,trx_count
108098,2019-06,999,기타,1.0,400000.0,1,0,0,490000,1
108099,2019-06,999,기타,1.0,500000.0,1,0,0,550000,1
108100,2019-06,999,기타,1.0,500000.0,6,0,0,500000,1
108101,2019-06,999,기타,1.0,500000.0,7,0,0,500000,1
108102,2019-06,999,기타,1.0,800000.0,more_20th_sell,0,0,850000,1


In [10]:
bunp_help_order_pivot = bunp_help_order_result.pivot_table(index='month_at', 
                                   columns=['sell_order_group', 'fraud_help'], 
                                   values='price_sum', 
                                   aggfunc='sum')

# save
save_path = '../csv/result/bunp_help_order_pivot.csv'
bunp_help_order_pivot.to_csv(save_path, index=True, mode='w', header=True)

bunp_help_order_pivot

sell_order_group           1.0                       2.0              \
fraud_help                 0.0         1.0           0.0         1.0   
month_at                                                               
2016-09           4.000000e+04         NaN           NaN         NaN   
2016-10           5.511628e+08   7756100.0  1.646074e+08   2188004.0   
2016-11           1.159941e+09  18765911.0  4.230646e+08   7051500.0   
2016-12           1.453151e+09  32006608.0  6.221984e+08  13989116.0   
2017-01           1.633309e+09  22208700.0  7.842651e+08  13008900.0   
2017-02           2.395090e+09  30514321.0  1.358209e+09  22763200.0   
2017-03           1.930203e+09  26687704.0  1.114922e+09  15839500.0   
2017-04           2.196520e+09  29938805.0  1.288252e+09  21901756.0   
2017-05           1.801183e+09  31041810.0  1.090110e+09  19526449.0   
2017-06           1.757238e+09  25025900.0  1.056609e+09  21226129.0   
2017-07           2.101746e+09  32244234.0  1.402611e+09  20136815.0   
2017-08           2.442193e+09  47346353.0  1.582748e+09  27903734.0   
2017-09           2.556874e+09  25608010.0  1.567720e+09  19137115.0   
2017-10           2.583500e+09  30489499.0  1.673100e+09  22905333.0   
2017-11           3.089514e+09  30406000.0  1.878775e+09  32053000.0   
2017-12           2.704945e+09  31058500.0  1.703044e+09  27254100.0   
2018-01           2.415223e+09  30521118.0  1.558058e+09  23757000.0   
2018-02           2.414451e+09  26519100.0  1.557845e+09  24930600.0   
2018-03           2.542896e+09  40408957.0  1.628828e+09  23020012.0   
2018-04           2.303647e+09  39363332.0  1.456501e+09  23517100.0   
2018-05           2.115524e+09  35368704.0  1.333674e+09  21986500.0   
2018-06           2.213046e+09  22534300.0  1.324296e+09  23438201.0   
2018-07           2.252023e+09  35500000.0  1.415841e+09  22825000.0   
2018-08           2.415995e+09  41690551.0  1.511294e+09  33418412.0   
2018-09           2.433583e+09  30964200.0  1.555568e+09  22454249.0   
2018-10           2.864795e+09  50265500.0  1.824567e+09  27524000.0   
2018-11           2.990250e+09  49790211.0  1.864082e+09  32331500.0   
2018-12           2.895536e+09  47400213.0  1.854824e+09  41135900.0   
2019-01           3.165319e+09  42069100.0  1.943271e+09  32162000.0   
2019-02           3.124617e+09  34718010.0  1.930571e+09  24434900.0   
2019-03           2.893189e+09  38946901.0  1.757580e+09  25204200.0   
2019-04           2.784008e+09  36702938.0  1.678538e+09  32188753.0   
2019-05           2.510045e+09  38887500.0  1.468337e+09  25413850.0   
2019-06           2.325821e+09  48583804.0  1.378757e+09  23124010.0   

sell_order_group           3.0                       4.0              \
fraud_help                 0.0         1.0           0.0         1.0   
month_at                                                               
2016-09                    NaN         NaN           NaN         NaN   
2016-10           7.816474e+07    429500.0  5.315832e+07    110500.0   
2016-11           2.313053e+08   3609000.0  1.542155e+08   1344300.0   
2016-12           3.720177e+08   7075800.0  2.511644e+08   3726500.0   
2017-01           4.875747e+08   6110280.0  3.462318e+08   5558300.0   
2017-02           8.847438e+08  13578500.0  6.684658e+08   9823000.0   
2017-03           7.671573e+08  10832600.0  5.824420e+08   5762500.0   
2017-04           9.309704e+08  15804945.0  6.797347e+08   9734811.0   
2017-05           7.999541e+08  12107400.0  6.407784e+08   6973217.0   
2017-06           7.241085e+08  10684000.0  5.935092e+08   6926577.0   
2017-07           1.026975e+09  17969204.0  7.686508e+08  13280423.0   
2017-08           1.162554e+09  23518004.0  9.132256e+08  15334111.0   
2017-09           1.177544e+09  12373501.0  9.372234e+08  10895599.0   
2017-10           1.247531e+09  15758121.0  1.012178e+09  12679811.0   
2017-11           1.418591e+09  18806311.0  1.094926e+09  13776499.0   
2017-12           1.246222e+09  1835093

In [11]:
bunp_help_order_result_5 = bunp_help_order_result[bunp_help_order_result['less_50,000'] == 1 ]

bunp_help_order_pivot_5 = bunp_help_order_result_5.pivot_table(index='month_at', 
                                   columns=['sell_order_group', 'fraud_help'], 
                                   values='price_sum', 
                                   aggfunc='sum')

# save
save_path = '../csv/result/bunp_help_order_pivot_5.csv'
bunp_help_order_pivot_5.to_csv(save_path, index=True, mode='w', header=True)

bunp_help_order_pivot_5

sell_order_group          1.0                     2.0                     3.0  \
fraud_help                0.0        1.0          0.0        1.0          0.0   
month_at                                                                        
2016-09               40000.0        NaN          NaN        NaN          NaN   
2016-10           159446874.0   754100.0   51028899.0   270004.0   27313359.0   
2016-11           282775571.0  1898000.0  119946179.0   570500.0   73113788.0   
2016-12           343833830.0  2345708.0  165356643.0  1023116.0  109182723.0   
2017-01           473958124.0  2776300.0  260929036.0  1563900.0  166689548.0   
2017-02           777529714.0  3721313.0  497880483.0  2291400.0  352109222.0   
2017-03           614094160.0  2287704.0  412754803.0  1939500.0  307286632.0   
2017-04           633670414.0  2766805.0  465268034.0  1993400.0  355863268.0   
2017-05           490217595.0  2095811.0  369913302.0  1751338.0  298635801.0   
2017-06           432472295.0  1893900.0  333539171.0  1357129.0  271523206.0   
2017-07           485308212.0  2261234.0  399499725.0  1644315.0  323406515.0   
2017-08           543708856.0  1602855.0  439726379.0  1575234.0  363069401.0   
2017-09           534231251.0  1697000.0  426843381.0  1657115.0  352274096.0   
2017-10           515088382.0  1698499.0  407732156.0  1438333.0  339950618.0   
2017-11           506568901.0  1137000.0  388729793.0  1318000.0  317180025.0   
2017-12           470869040.0  1341000.0  368032892.0  1298100.0  304416177.0   
2018-01           505171101.0  1561118.0  393818286.0  1587000.0  328360534.0   
2018-02           474804836.0  1729600.0  376192814.0  1299600.0  325051417.0   
2018-03           477069632.0  1097557.0  383660335.0  1328604.0  317477896.0   
2018-04           398733464.0  1924610.0  316318743.0  1148100.0  266720759.0   
2018-05           358279966.0  1538704.0  286569728.0  1569500.0  242207937.0   
2018-06           341299192.0  1214800.0  269741268.0  1106900.0  225529475.0   
2018-07           354605803.0  1281000.0  275161310.0   840000.0  234678073.0   
2018-08           375003643.0  1867551.0  297641723.0  1010611.0  246700772.0   
2018-09           365126099.0  1521000.0  293541531.0  1011249.0  253917101.0   
2018-10           401523879.0  1588500.0  322668092.0  1529000.0  276952378.0   
2018-11           371132790.0  1496211.0  302613425.0  1714000.0  251661850.0   
2018-12           380013565.0  1757113.0  296102315.0  1556900.0  247111718.0   
2019-01           455524698.0  1668100.0  356419294.0  1449000.0  296118391.0   
2019-02           460518060.0  1162010.0  362521207.0  1276900.0  303927948.0   
2019-03           420320107.0  1505001.0  322038329.0  1249400.0  273869611.0   
2019-04           366038627.0  1727438.0  288201326.0  1371754.0  238097794.0   
2019-05           333418928.0  1353000.0  263082895.0  1473800.0  218960364.0   
2019-06           302869954.0  1193804.0  231660963.0  1366510.0  195833459.0   

sell_order_group                     4.0                     5.0             \
fraud_help              1.0          0.0        1.0          0.0        1.0   
month_at                                                                      
2016-09                 NaN          NaN        NaN          NaN        NaN   
2016-10            199500.0   18103149.0   110500.0   12602395.0    72900.0   
2016-11            331000.0   52767712.0   225300.0   40792612.0   218000.0   
2016-12            565800.0   79420770.0   227000.0   62195519.0   140800.0   
2017-01            898280.0  122874582.0   670300.0   95931337.0   357611.0   
2017-02           1698500.0  264687132.0  1063000.0  212519254.0   843901.0   
2017-03           1301600.0  238277191.0   989500.0  195172991.0   713000.0   
2017-04           1514045.0  283481635.0  1306811.0  234342255.0  1398000.0   
2017-05           1562400.0  246754662.0   958217.0  211638998.0   992377.0   
2017-06           1038000.0  228051112.0   621577.0  1